In [204]:
import pandas as pd
import numpy as np
from funcs import *
import unidecode
import basedosdados as bd

In [205]:
# 2.4. DETERMINANTE MERCADO

mercado = {}

In [206]:
# 2.4.1. Subdeterminante Desenvolvimento Econômico

subdet = 'Desenvolvimento Econômico'

In [207]:
# 2.4.1.1. Indicador Índice de Desenvolvimento Humano

ind = pd.read_excel('Arquivos ICE - 23/Ind_Originais_ICE_2022.xlsx', header=[i for i in range(6)], index_col=[0,1])
sub_desenveco = ind['Mercado', 'Desenvolvimento Econômico', 'Índice de Desenvolvimento Humano']
sub_desenveco.columns = sub_desenveco.columns.droplevel([0,1])
sub_desenveco.columns.values[0] = 'Índice de Desenvolvimento Humano'

In [215]:
# 2.4.1.2. Indicador Crescimento Médio Real do PIB

pib = pd.read_excel('DETERMINANTE MERCADO/tabela5938.xlsx', header=3).head(-1)
pib.columns.values[0] = 'Município'
pib['UF'] = pib['Município'].apply(lambda x: x.split('(')[-1][:2])
pib['Município'] = pib['Município'].apply(lambda x: x.split('(')[0].strip())
pib = pib.set_index(['Município', 'UF'])

# O DEFLATOR DO PIB UTILIZADO É DA FONTE IPEADATA E NÃO IBGE

deflator = pd.DataFrame([{
    '2016':8.10,
    '2017':3.67,
    '2018':4.49,
    '2019':4.22
}]).T

In [223]:
(deflator +100)/(deflator.loc['2019',0]+100)

,0
2016,1.037229
2017,0.994723
2018,1.002591
2019,1.000000


In [209]:
# 2.4.1.3. Indicador Número de Empresas Exportadoras com Sede na Cidade

emp_exp = pd.read_excel('DETERMINANTE MERCADO/EMPRESAS_CADASTRO_2020.xlsx', header=7)

convert = lambda x: unidecode.unidecode(x.upper())
n_exp = {n:len(emp_exp.groupby(['MUNICÍPIO', 'UF']).get_group(tuple([convert(i) for i in n]))) for n in sub_desenveco.index}
n_exp = pd.DataFrame([n_exp],index=['n_exp']).T
n_exp.index = pd.MultiIndex.from_tuples(n_exp.index, names=['Município', 'UF'])

variaveis = ('COUNT(quantidade_vinculos_ativos), id_municipio')

## Montando a query
base = '`basedosdados.br_me_rais.microdados_estabelecimentos`'
project_id = "trim-descent-346220"
query = f"SELECT id_municipio, COUNT(quantidade_vinculos_ativos) FROM {base} WHERE ano = 2020 AND quantidade_vinculos_ativos > 0 GROUP BY id_municipio"

## Importando o data lake
df_rais = bd.read_sql(query=query, billing_project_id=project_id).set_index('id_municipio')

cod = pd.read_excel('DETERMINANTE MERCADO/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')
cod = cod.rename(columns={'Código Município Completo':'id_municipio'})
cod['id_municipio'] = cod['id_municipio'].apply(str)
cod = cod[['Nome_Município','Nome_UF','id_municipio']].set_index('id_municipio')

n_rais = pd.merge(cod, df_rais, left_index=True, right_index=True).rename(columns={
    'Nome_Município':'Município',
    'Nome_UF':'UF',
    'f0_':'n_rais'
}).set_index(['Município', 'UF'])

ratio = pd.merge(n_rais, n_exp, left_index=True, right_index=True)
ratio['ratio'] = ratio['n_exp']/ratio['n_rais']

sub_desenveco['Número de Empresas Exportadoras com Sede na Cidade'] = ratio['ratio']

Downloading: 100%|████████████████████████████████████████████████████████████| 5570/5570 [00:00<00:00, 10247.33rows/s]


In [210]:
sub_desenveco

,UF,Índice de Desenvolvimento Humano,Número de Empresas Exportadoras com Sede na Cidade
Rio de Janeiro,RJ,0.799,0.005925
São Paulo,SP,0.805,0.013096
Florianópolis,SC,0.847,0.005943
Curitiba,PR,0.823,0.010232
Joinville,SC,0.809,0.017927
...,...,...,...
São Vicente,SP,0.768,0.004187
São João de Meriti,RJ,0.719,0.001978
Ribeirão das Neves,MG,0.684,0.003631
Itaquaquecetuba,SP,0.714,0.029337


In [ ]:
# 2.4.2. Subdeterminante Clientes Potenciais

subdet = 'Desenvolvimento Econômico'